In [ ]:
import sys
import os  #برای کار با فایل‌ها و مسیرها
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer #تبدیل متن به ویژگی‌های عددی
#  در مدل ما، از TfidfVectorizer استفاده کردیم که مربوط به متن هستش و
#  مقدارهایی مثل 0.32، 0.05، ... تولید می‌کنه (یعنی غیرباینریه).
from sklearn.model_selection import train_test_split #جداسازی دیتا به تست و ترین
from sklearn.metrics import classification_report # ارزیابی مدل
import joblib #ذخیره مدل


---
# main file

In [ ]:
# ایمپورت کردن مدل‌ها به صورت داینامیک
# زمانی که پروژه ما چند مدل را یاد خواهد گرفت
model_name = sys.argv[1] if len(sys.argv) > 1 else "nb"  # پیش‌فرض: Naive Bayes
#با استفاده از sys.argv، از بیرون فایل هم می‌توانی مدل را تعیین کنی.
#این خط تعیین می‌کنه که مدل پیش‌فرض چی باشه، نه اینکه مدل واقعاً ایمپورت یا استفاده شده باشه تا اینکه به خطهای بعدی برسه.

# بدون این خط میشد به صورت دستی هر بار نام مدل را قرار بدیم مثلا
# model_name = "svm"
# اما در این صورت نمی‌تونی از بیرون (یعنی از طریق ترمینال) مدل رو تغییر بدی
# و مجبور می‌شی هر بار بیای توی فایل main.py و دستی کد رو عوض کنی.

# به زبان ساده می‌گوید:
#     اگه کاربر مدلی را وارد کرده (یعنی آرگومان دوم وجود دارد)، از آن استفاده کن.
#     وگرنه، مدل پیش‌فرض را بذار 'nb' (یعنی Naive Bayes).

# #sys.argv[0] → نام فایل اجرایی: 'main.py'
# sys.argv[1] → اولین آرگومان نام مدل ما است: 'svm'

 # چرا > 1؟
# چون همیشه:
#     sys.argv[0] → نام فایل پایتون هست.
#     پس اگر فقط برنامه اجرا بشه (بدون هیچ آرگومانی)، طول لیست sys.argv فقط ۱ هست:
# ['main.py']
# بنابراین:
#     وقتی طول sys.argv بیشتر از 1 باشه، یعنی حداقل یک آرگومان بعد از اسم فایل هم وارد شده.

if model_name == "nb":
    from models.train_nb_model import train_model
    #  اگر فایل train_nb_model.py وجود نداشته باشه
    # که پیش‌فرض هم همینه، اون‌وقت در زمان اجرا ارور می‌گیری (مثلاً: ModuleNotFoundError).
elif model_name == "svm":
    from models.train_svm_model import train_model
elif model_name == "lr":
    from models.train_lr_model import train_model
else:
    raise ValueError("Unsupported model")

In [ ]:
# تابع برای خواندن ایمیل‌ها
# # تمام فایلهای متنی را میخواند و داخل لیستی میریزد
def load_emails_from_folder(folder):
    emails = []
    for filename in os.listdir(folder):
        path = os.path.join(folder, filename)
        with open(path, 'r', encoding='latin-1') as f:
            emails.append(f.read())
    return emails
# خواندن ایمیل‌های اسپم و معمولیاز پوشه‌هایشان در data/ می‌خواند.
spam_emails = load_emails_from_folder('data/spam')
ham_emails = load_emails_from_folder('data/easy_ham')

In [ ]:
# ساخت دیتافریم و دادن لیبل به انها
# # ستون text شامل متن ایمیل‌ها.
# # ستون label: عدد ۱ برای اسپم، عدد ۰ برای ایمیل‌های معمولی.
df = pd.DataFrame({'text': spam_emails + ham_emails,
                   'label': [1]*len(spam_emails) + [0]*len(ham_emails)})
# #فرض کن 3 ایمیل اسپم داریم
# # اون‌وقت len(spam_emails) می‌شه 3. حالا: [1] * 3
# # مساوی می‌شه با: [1, 1, 1]
# # # یعنی برای ۳ ایمیل اسپم، ۳ عدد ۱ تولید کردیم. همین کار برای ایمیل‌های معمولی:
# # [0] * len(ham_emails)
# # در نتیجه:
# # y = [1, 1, 1, 0, 0, 0]

In [ ]:
# تبدیل محتوای ایمیل به عدد
vectorizer = TfidfVectorizer(stop_words='english', max_features=1000)
# #می‌گوید که کلمات توقف (stop words) زبان انگلیسی را نادیده بگیرد.مانند "the"، "a"، "is"، "are" و غیره.
# با حذف این کلمات، تمرکز مدل بر روی کلمات مهم‌تر خواهد بود.
# #فقط ۱۰۰۰ کلمه‌ی مهم‌تر و پرتکرارتر را از بین تمام کلمات موجود در همه ایمیل‌ها نگه دار و بقیه را نادیده بگیر
X = vectorizer.fit_transform(df['text'])
#fit() یعنی: از روی تمام متن‌ها، کلمات پرتکرار و مهم (طبق TF-IDF) رو یاد بگیر
# و برای هر کلمه، یک عدد شاخص مشخص کن.
# transform() یعنی: حالا بیایم هر متن رو تبدیل به بردار عددی کنیم با توجه به آن چیزی که در fit() یاد گرفتیم.
y = df['label']

In [ ]:
#دسته بندی دیتا به تست و ترین
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
# آموزش مدل
model = train_model(X_train, y_train)

In [ ]:
#  پیش‌بینی روی داده‌های تست و چاپ گزارش عملکرد مدل.
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

In [ ]:
# # اطمینان از وجود فولدر models
os.makedirs('models', exist_ok=True)


In [ ]:
# ذخیره مدل و بردار‌ساز
joblib.dump(model, f'models/{model_name}_classifier.pkl')
joblib.dump(vectorizer, 'models/vectorizer.pkl')



In [ ]:
"""
مهم:
1. برای اجرا به فایل main میرویم
2. با توجه به اینکه خروجی چه مدلی را میخواهیم یکی از کدهای زیر را در ترمینال میزنیم
python main.py nb    
python main.py svm
python main.py lr

"""

In [ ]:
#مدل‌های مختلف شما در پوشه models/ ذخیره خواهند شد.
# به عنوان مثال، اگر مدل SVM را انتخاب کنید، فایل مدل ذخیره‌شده به نام svm_classifier.pkl خواهد بود.


---
# آموزش مدلهای مختلف به ماشین


In [ ]:
# train_nb_model.py
#################################
# Training code goes here
#🔹 معمولاً آموزش مدل از اینجا جدا می‌شود،
# مثلاً اگر بخواهید فقط مدل را دوباره آموزش دهید بدون اجرای بقیه چیزها.
# 🔹 بهتره برای هر مدل یک فایل جدید آموزشی داشته باشی، مثل:
#     train_nb.py (برای Naive Bayes)
#     train_lr.py (برای Logistic Regression)
#     train_svm.py (برای SVM)

#################################
# train_nb_model.py
from sklearn.naive_bayes import MultinomialNB

def train_model(X_train, y_train):
    model = MultinomialNB()
    model.fit(X_train, y_train)
    return model


"""
مهم:
1. برای اجرا به فایل main میرویم
2. کد زیر را در ترمینال میزنیم و به صورت مستقیم علامت پلی را نیمزنیم
python main.py nb

"""


In [ ]:
# train_svm_model.py

from sklearn.svm import SVC

def train_model(X_train, y_train):
    model = SVC(kernel='linear')
    model.fit(X_train, y_train)
    return model


"""
مهم:
1. برای اجرا به فایل main میرویم
2. کد زیر را در ترمینال میزنیم و به صورت مستقیم علامت پلی را نیمزنیم
python main.py svm

"""

#مدل‌های مختلف شما در پوشه models/ ذخیره خواهند شد.
# به عنوان مثال، اگر مدل SVM را انتخاب کنید، فایل مدل ذخیره‌شده به نام svm_classifier.pkl خواهد بود.


In [ ]:
# train_lr_model.py
from sklearn.linear_model import LogisticRegression

def train_model(X_train, y_train):
    model = LogisticRegression(max_iter=1000)
    model.fit(X_train, y_train)
    return model

"""
مهم:
1. برای اجرا به فایل main میرویم
2. کد زیر را در ترمینال میزنیم و به صورت مستقیم علامت پلی را نیمزنیم
python main.py lr

"""

---
# ارزیابی


In [ ]:
# evaluate_nb_model.py

import os
from sklearn.metrics import classification_report, confusion_matrix

def evaluate_nb_model(model, X_test, y_test):
    y_pred = model.predict(X_test)

    report = classification_report(y_test, y_pred)
    matrix = confusion_matrix(y_test, y_pred)

    # چاپ در کنسول
    print("Naive Bayes Classification Report:")
    print(report)
    print("Naive Bayes Confusion Matrix:")
    print(matrix)

    # ذخیره در فایل
    output_path = os.path.join("evaluation_outputs", "nb_report.txt")
    with open(output_path, 'w') as f:
        f.write("Naive Bayes Classification Report:\n")
        f.write(report + "\n\n")
        f.write("Naive Bayes Confusion Matrix:\n")
        f.write(str(matrix))

"""
برای خروجی این فایل باید از فایل evaluate_models.py خروجی گرفته شود  
"""

In [ ]:
# evaluate_lr_model.py




import os
from sklearn.metrics import classification_report, confusion_matrix

def evaluate_lr_model(model, X_test, y_test):
    y_pred = model.predict(X_test)

    report = classification_report(y_test, y_pred)
    matrix = confusion_matrix(y_test, y_pred)

    print("Logistic Regression Classification Report:")
    print(report)
    print("Logistic Regression Confusion Matrix:")
    print(matrix)

    output_path = os.path.join("evaluation_outputs", "lr_report.txt")
    with open(output_path, 'w') as f:
        f.write("Logistic Regression Classification Report:\n")
        f.write(report + "\n\n")
        f.write("Logistic Regression Confusion Matrix:\n")
        f.write(str(matrix))


"""
برای خروجی این فایل باید از فایل evaluate_models.py خروجی گرفته شود  
"""

In [ ]:
# # evaluate_svm_model.py
# # ارزیابی مدل اس وی ام
#
# from sklearn.metrics import classification_report, confusion_matrix
#
# def evaluate_svm_model(model, X_test, y_test):
#     y_pred = model.predict(X_test)
#
#     # گزارش طبقه‌بندی
#     print("SVM Classification Report:")
#     print(classification_report(y_test, y_pred))
#
#     # ماتریس سردرگمی
#     print("SVM Confusion Matrix:")
#     print(confusion_matrix(y_test, y_pred))
#########
import os
from sklearn.metrics import classification_report, confusion_matrix

def evaluate_svm_model(model, X_test, y_test):
    y_pred = model.predict(X_test)

    report = classification_report(y_test, y_pred)
    matrix = confusion_matrix(y_test, y_pred)

    # چاپ در کنسول
    print("SVM Classification Report:")
    print(report)
    print("SVM Confusion Matrix:")
    print(matrix)

    # ذخیره در فایل
    output_path = os.path.join("evaluation_outputs", "svm_report.txt")
    with open(output_path, 'w') as f:
        f.write("SVM Classification Report:\n")
        f.write(report + "\n\n")
        f.write("SVM Confusion Matrix:\n")
        f.write(str(matrix))

"""
برای خروجی این فایل باید از فایل evaluate_models.py خروجی گرفته شود  
"""

In [ ]:
# compare_models.py
# فایل ارزیابی ای است تا سه مدل Naive Bayes و  SVM و Logistic Regression را با هم مقایسه کند


import os
import joblib
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

#  رسم نمودار میله‌ای برای مقایسه مدل‌ها
def plot_model_comparison(results, save_path):
    labels = ["Accuracy", "Precision", "Recall", "F1-Score"]
    model_names = [r[0] for r in results]
    #این یک لیست‌ساز (list comprehension) است
    #برای هر عنصر r، اولین آیتم آن (با اندیس 0) انتخاب می‌شود.
    #در نهایت، یک لیست جدید به نام model_names ایجاد می‌شود که شامل نام تمام مدل‌های موجود در results است.

    scores = [r[1:] for r in results]
    # یک برش (slice) از آن ایجاد می‌شود که شامل تمام آیتم‌ها از اندیس 1 به بعد است.
    #در نهایت یک لیست جدید به نام scores ایجاد می‌شود که در آن هر عنصر، لیستی از نمرات ارزیابی برای یک مدل خاص است.
    scores = np.array(scores)
    # برای تبدیل لیست scores به یک آرایه NumPy استفاده می‌شود.
    # آرایه‌های NumPy برای انجام محاسبات عددی کارآمدتر هستند و برای رسم نمودار با matplotlib مناسب‌ترند.

    x = np.arange(len(labels))
    #با استفاده از np.arange یک آرایه NumPy از اعداد صحیح ایجاد می‌شود.

    width = 0.25 # این مقدار برای تعیین پهنای میله‌های نمودار
    fig, ax = plt.subplots(figsize=(10, 6))
    for i in range(len(model_names)):
        ax.bar(x + i * width, scores[i], width, label=model_names[i])
        #x + i * width: موقعیت x برای میله‌های مربوط به مدل فعلی (i).
        # با اضافه کردن i * width، میله‌های مربوط به هر مدل کمی به سمت راست جابجا می‌شوند تا از هم جدا باشند.

    ax.set_ylabel('Score') #عنوان کلی برای محور y است که 'Score' را گذاشته.
    ax.set_title('Model Comparison')
    ax.set_xticks(x + width)
    #مکان قرارگیری نشانه‌های محور x تنظیم می‌شود.
    # از x + width استفاده می‌شود تا نشانه‌ها در وسط گروه‌های میله‌ها قرار بگیرند.

    ax.set_xticklabels(labels) #برچسب‌های مربوط به نشانه‌های محور x با استفاده از لیست labels تنظیم می‌شو
    ax.legend() #راهنمای نمودار
    ax.grid(axis='y', linestyle='--', alpha=0.7)

    plt.tight_layout()
    #یک تابع که به طور خودکار فاصله‌بندی بین عناصر نمودار (مانند عنوان، برچسب‌ها و راهنما) را تنظیم می‌کند تا از همپوشانی جلوگیری شود.

    plt.savefig(save_path)
    #نمودار تولید شده در فایلی با مسیری که در پارامتر save_path مشخص شده است، ذخیره می‌شود.

    plt.close()
    #شکل (figure) مربوط به نمودار بسته می‌شود تا منابع سیستم آزاد شوند.

    print(f"✅ نمودار ذخیره شد: {save_path}")


#  مقایسه مدل‌ها + تولید خروجی متنی و تصویری
def compare_models(X_test, y_test, models_dir):
    model_files = {
        "SVM": "svm_classifier.pkl",
        "Naive Bayes": "nb_classifier.pkl",
        "Logistic Regression": "lr_classifier.pkl"
    }

    results = []

    for model_name, filename in model_files.items():
        try:
            model_path = os.path.join(models_dir, filename)
            model = joblib.load(model_path)
            y_pred = model.predict(X_test)

            acc = accuracy_score(y_test, y_pred)
            prec = precision_score(y_test, y_pred)
            rec = recall_score(y_test, y_pred)
            f1 = f1_score(y_test, y_pred)

            results.append((model_name, acc, prec, rec, f1))

            print(f"\n📊 {model_name} Scores:")
            print(f"Accuracy: {acc:.4f}")
            print(f"Precision: {prec:.4f}")
            print(f"Recall: {rec:.4f}")
            print(f"F1-Score: {f1:.4f}")

        except Exception as e:
            print(f"⚠️ خطا در بارگذاری یا ارزیابی {model_name}: {e}")

    #  مسیر خروجی‌ها: داخل src/evaluation_outputs
    base_dir = os.path.dirname(os.path.abspath(__file__))
    outputs_dir = os.path.join(base_dir, 'evaluation_outputs')
    os.makedirs(outputs_dir, exist_ok=True)

    #  گزارش متنی
    report_path = os.path.join(outputs_dir, 'compare_models_report.txt')
    with open(report_path, 'w', encoding='utf-8') as f:
        f.write("Model Comparison:\n")
        f.write(f"{'Model':<20}{'Accuracy':<10}{'Precision':<10}{'Recall':<10}{'F1-Score':<10}\n")
        for model_name, acc, prec, rec, f1 in results:
            line = f"{model_name:<20}{acc:<10.2f}{prec:<10.2f}{rec:<10.2f}{f1:<10.2f}\n"
            f.write(line)
            print(line, end='')

    print(f"\n✅ گزارش متنی ذخیره شد: {os.path.abspath(report_path)}")

    # 🖼️ ذخیره نمودار
    chart_path = os.path.join(outputs_dir, 'compare_models_chart.png')
    plot_model_comparison(results, chart_path)

#  تابع main فقط برای تست (اختیاری)
def main():
    import pandas as pd
    from sklearn.model_selection import train_test_split

    data_path = os.path.join(os.path.dirname(__file__), '..', 'data', 'processed_data.csv')
    df = pd.read_csv(data_path)

    X = df.drop('label', axis=1)
    y = df['label']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    models_dir = os.path.join(os.path.dirname(__file__), '..', 'models')
    compare_models(X_test, y_test, models_dir)

if __name__ == "__main__":
    main()

"""
برای خروجی این فایل باید از فایل evaluate_models.py خروجی گرفته شود  
"""

In [ ]:
# evaluate_models.py
##################
from evaluate_svm_model import evaluate_svm_model
from evaluate_nb_model import evaluate_nb_model
from evaluate_lr_model import evaluate_lr_model
import os
from sklearn.model_selection import train_test_split
import pandas as pd
from compare_models import compare_models
import joblib

from sklearn.feature_extraction.text import TfidfVectorizer
# ___________________________________

# مسیر نسبی به فولدر data
base_dir = os.path.dirname(os.path.abspath(__file__))
#os.path.abspath(__file__): این قسمت، مسیر مطلق (کامل) فایل جاری (evaluate_models.py) را در سیستم عامل برمی‌گرداند.

spam_dir = os.path.join(base_dir, '..', 'data', 'spam')
#os.path.join(...): این تابع برای ساختن یک مسیر فایل یا دایرکتوری به صورت هوشمندانه و سازگار با سیستم عامل‌های مختلف استفاده می‌شود. این تابع به طور خودکار از جداکننده مناسب مسیر (مثل / در لینوکس و \ در ویندوز) استفاده می‌کند.
# '..': این علامت در مسیر به معنای "رفتن به دایرکتوری والد" است.
# 'data', 'spam', 'easy_ham': اینها نام دایرکتوری‌ها یا فایل‌ها هستند.

ham_dir = os.path.join(base_dir, '..', 'data', 'easy_ham')

# تابع برای خواندن ایمیل‌ها
def load_emails_from_folder(folder):
    emails = []
    for filename in os.listdir(folder):
        path = os.path.join(folder, filename)
        with open(path, 'r', encoding='latin-1') as f:
            emails.append(f.read())
    return emails

# خواندن ایمیل‌ها
spam_emails = load_emails_from_folder(spam_dir)
ham_emails = load_emails_from_folder(ham_dir)
df = pd.DataFrame({'text': spam_emails + ham_emails,
                   'label': [1]*len(spam_emails) + [0]*len(ham_emails)})


models_dir = os.path.join(base_dir, '..', 'models')
vectorizer_path = os.path.join(models_dir, 'vectorizer.pkl')

# تقسیم داده‌ها به آموزش و تست بدون فیت
vectorizer = joblib.load(vectorizer_path)
X = vectorizer.transform(df['text'])   # فقط transform (بدون fit)
y = df['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# ___________________________________

def main():
    try:
        print("Evaluating SVM Model:")
        svm_path = os.path.join(models_dir, 'svm_classifier.pkl')
        model_svm = joblib.load(svm_path)
        evaluate_svm_model(model_svm, X_test, y_test)
    except Exception as e:
        print(f"Error evaluating SVM model: {e}")

    try:
        print("\nEvaluating Naive Bayes Model:")
        nb_path = os.path.join(models_dir, 'nb_classifier.pkl')
        model_nb = joblib.load(nb_path)
        evaluate_nb_model(model_nb, X_test, y_test)
    except Exception as e:
        print(f"Error evaluating Naive Bayes model: {e}")

    try:
        print("\nEvaluating Logistic Regression Model:")
        lr_path = os.path.join(models_dir, 'lr_classifier.pkl')
        model_lr = joblib.load(lr_path)
        evaluate_lr_model(model_lr, X_test, y_test)
    except Exception as e:
        print(f"Error evaluating Logistic Regression model: {e}")

    print("\nComparing all models:")
    compare_models(X_test, y_test, models_dir)  # اینجا منتقل شود داخل main



#_________________________________________

# مسیر فولدر evaluation_outputs
outputs_dir = "evaluation_outputs"

# اگر فولدر evaluation_outputs وجود نداشت، آن را ایجاد کن
if not os.path.exists(outputs_dir):
    os.makedirs(outputs_dir)
    print(f"Folder '{outputs_dir}' created.")
else:
    print(f"Folder '{outputs_dir}' already exists.")


if __name__ == "__main__":
    main()


## ____________________________________________
"""
1. این فایل را میتوان با دکمه پلی خروجی گرفت
2. این فایل خروجی فایلهای زیر را میگیرد و در پوشه src/evaluation_outputs ذخیره میکند
evaluate_lr_model.py و evaluate_nb_model و evaluate_svm_model.py و compare_models.py  
"""

---
# predict پیش بینی

In [ ]:
# predict.py


#🔹 برای استفاده از مدل روی ایمیل جدید:
# 🔹 متن ایمیل جدید را عددی می‌کند و نتیجه (۰ یا ۱) را برمی‌گرداند.
# predict.py: فقط یک ماژول کمکی است که تابع predict_email(text) را تعریف می‌کند.
# و این تابع یک متن می‌گیرد و نتیجه‌ی پیش‌بینی را (۰ یا ۱) برمی‌گرداند. خودش به‌تنهایی چیزی چاپ نمی‌کند.
#  اما فایل predict_email.py: یک اسکریپت قابل اجرا است که ایمیل‌های نمونه را به تابع predict_email() می‌دهد و نتیجه را چاپ می‌کند.

# توضیح بیشتر
#  پس:
#     predict.py = تابع منطقی برای استفاده در جاهای دیگر.
#     predict_email.py = اجرای عملی و نمونه با چاپ خروجی.


from joblib import load

def predict_email(model_name, text):
    # بارگذاری مدل و بردار‌ساز بر اساس نام مدل
    model = load(f'models/{model_name}_classifier.pkl')
    vectorizer = load('models/vectorizer.pkl')
    # چون هر متن رو به یک بردار ویژگی (feature vector) تبدیل می‌کنه.
    # و در ریاضیات و یادگیری ماشین، به چنین لیست‌هایی از اعداد «بردار» گفته می‌شه.

    # تبدیل متن ایمیل به ویژگی‌های عددی
    X = vectorizer.transform([text])

    #پیش بینی
    prediction = model.predict(X)[0]
    # [0]
    # این یعنی:
    #     «از بین همه پیش‌بینی‌ها، فقط اولین (و تنها) مورد را بردار.»
    # چرا؟ چون ما فقط یک ایمیل را داریم پیش‌بینی می‌کنیم.
    # ولی چون vectorizer.transform([text]) خروجی‌اش یک آرایه با یک نمونه است،
    # مدل هم خروجی‌اش یک آرایه با یک مقدار خواهد بود.

    # در بیشتر مدلهای دو کلاسه model.predict(X) یک آرایه به نام array([1])  یا array([0])برمیگردونه
    #اگر array([0]) باشد، اولین عنصر صفر است و اگر array([1]) باشد، اولین عنصر یک است.
    # یعنی اگر ایمیل اسپم است، خروجی به شکل array([1]) خواهد بود و..


    # بازگشت نتیجه پیش‌بینی
    return "This email **IS SPAM** ❌ " if prediction == 1 else "This email is **NOT SPAM** ✅"


In [ ]:
# predict_email.py
# # 🔹 بعد از ساخت و اموزش و کارهای اصلی
# # این فایل جدید را ساختیم تا برایمان بگوید که ایمیل های دستی زیر اسپم هستند یا خیر
# #         "Congratulations! You won a free ticket to Bahamas!",
# #         "Hi John, can we meet at the cafe tomorrow?",
# #         "URGENT: Update your banking information immediately.",
# #         "Here is the report you asked for. Let me know your thoughts."

##______________________
# # توضیح بیشتر:
# # آیا نمی‌شد فایلهای   predict_email.py و  predict.py را یکی کرد؟
# #
# # 1. فایل predict.py قراره مثل مغزِ پیش‌بینی مدل باشه. می‌تونه توسط Flask، تست، یا هر اسکریپت دیگه import بشه.
# # ولی predict_email.py فقط یه تست‌کننده ساده است. فقط برای انسان‌ها، نه ماشین یا سیستم.
#
# # 2اگر این دو فایل را یکی کنیم از نظر طراحی پروژه اشتباه است. اگه این کارو بکنی:
# #     هر بار که یه فایل کدی import predict کنه، کل اون ایمیل‌های تست هم اجرا می‌شن! ❌
# #     فایل سنگین، درهم، و غیرقابل نگهداری میش



# predict_email.py

from predict import predict_email

if __name__ == '__main__':
    emails = [
        "Congratulations! You won a free ticket to Bahamas!",
        "Meeting confirmed at 3pm with the HR team.",
        "Claim your prize now! Click here."
    ]

    model_name = 'svm'  # می‌توانید مدل را به 'svm', 'nb', یا 'lr' تغییر دهید.

    for i, email in enumerate(emails):
        print(f"Sample {i+1}:\n{email}")
        print(predict_email(model_name, email))
        print("-" * 30)

#____________________________________________
# # برای اجرای کد:
# # در ترمینال وارد پوشه پروژه شو و بنویس:
"""
 python src/predict_email.py
 
 """
# # چون وقتی با دکمه ▶️ پلی اجرا می‌کنی، PyCharm فایل رو اجرا می‌کنه
# ولی دایرکتوری جاری (Current Working Directory) ممکنه پوشه‌ی src باشه.
# # فایل‌های مدل شما (models/spam_classifier.pkl) در پوشه‌ی بالاتر هستند، بنابراین پیدا نمی‌شن.


In [ ]:
# predict_from_file.py

from joblib import load
from predict import predict_email


def predict_emails_from_file(filepath, model_name):
    # خواندن محتوای فایل
    with open(filepath, 'r', encoding='utf-8') as f:
        content = f.read()

    # تقسیم ایمیل‌ها
    emails = content.split('---')

    # پیش‌بینی برای هر ایمیل
    for i, email in enumerate(emails):
        email = email.strip()
        if not email:
            continue

        # استفاده از تابع predict_email برای پیش‌بینی ایمیل
        result = predict_email(model_name, email)

        # نمایش نتیجه
        print(f"\nEmail {i + 1}:")
        print(email)
        print("Result:", result)


if __name__ == "__main__":
    model_name = 'svm'  # مدل مورد نظر را انتخاب کنید: 'svm', 'nb', یا 'lr'
    predict_emails_from_file("test_emails.txt", model_name)

## ____________________________________________
# برای اجرای کد:
# در ترمینال وارد پوشه پروژه شو و بنویس:
"""
python src/predict_from_file.py
"""
# چون وقتی با دکمه ▶️ پلی اجرا می‌کنی، PyCharm فایل رو اجرا می‌کنه ولی دایرکتوری جاری (Current Working Directory) ممکنه پوشه‌ی src باشه.
# فایل‌های مدل شما (models/spam_classifier.pkl) در پوشه‌ی بالاتر هستند، بنابراین پیدا نمی‌شن.
